# mpl_bokeh_comparison
---
Using GeoViews, this notebook plots the same ashfall model with the Matplotlib backend versus the Bokeh backend.

Though it's a bit tricky to see, there is a small reduction in grid resolution for the Bokeh backend plot that occurs when GeoViews re-grids the model using cartopy's [`warp_array`](https://github.com/SciTools/cartopy/blob/master/lib/cartopy/img_transform.py) function. The re-gridding process is necessary for the Bokeh backend since Bokeh only has support for plotting [rectangular images with evenly-spaced axes](https://bokeh.pydata.org/en/latest/docs/reference/models/glyphs/image.html#) (at the time of writing). With the Matplotlib backend, on the other hand, we can take advantage of how closely Matplotlib and cartopy work together to project all sorts of wildly warped images, such as [this one](http://scitools.org.uk/cartopy/docs/v0.15/matplotlib/advanced_plotting.html#block-plots).

In [ ]:
# SPECIFY GLOBAL PARAMS

import numpy as np
import holoviews as hv
import geoviews as gv
import cartopy.crs as ccrs
import cartopy.feature as cf
import colorcet as cc
from bokeh.models.renderers import GlyphRenderer
from bokeh.models.glyphs import Patches

import sys
sys.path.insert(0, '../')
from vis_tools import read_hysplit_netcdf

###############################################
# SPECIFY: file name and path for HYSPLIT model
FILENAME = '../18042918_taupo_15.0_0.01.nc'

# SPECIFY:
ASH_MIN = 10**-2  # min ash colorbar cutoff
ASH_MAX = 10**2  # max ash colorbar cutoff
###############################################
       
model = read_hysplit_netcdf(FILENAME).isel(time=-1)
model['total_deposition'].values = np.log10(model['total_deposition'].values)  # manually take the log

gv_ds = gv.Dataset(model, crs=ccrs.PlateCarree())
gv_ds = gv_ds.redim.range(total_deposition=(np.log10(ASH_MIN), np.log10(ASH_MAX)))

STAMEN_TERRAIN = 'http://tile.stamen.com/terrain/{z}/{x}/{y}.jpg'
CMAP = cc.b_linear_kry_5_98_c75[::-1]
TITLE = FILENAME.split('/')[-1]

geoms = cf.GSHHSFeature(scale='i', levels=[1, 2]).intersecting_geometries([172, 179, -42, -34])
COASTLINES = gv.Feature(cf.ShapelyFeature(geoms, crs=ccrs.PlateCarree()))

GRATICLES = gv.Feature(cf.NaturalEarthFeature(category='physical', name='graticules_1', scale='110m'))

In [ ]:
# MATPLOTLIB BACKEND

hv.extension('matplotlib')
m_fig = gv.WMTS(STAMEN_TERRAIN) * gv_ds.to(gv.Image, ['lon', 'lat']) * COASTLINES * GRATICLES
       
def resize(plot, element):    
    plot.handles['fig'].set_size_inches(16,8)    
    
m_plot_opts = {'Image': {'style': dict(cmap=CMAP),
                          'plot': dict(apply_ranges=True, title_format=TITLE,
                                       projection=ccrs.GOOGLE_MERCATOR, final_hooks=[resize])},
             'Feature': {'style': dict(edgecolor='k', facecolor='none'),
                          'plot': dict(projection=ccrs.GOOGLE_MERCATOR)}
              }             
m_fig.opts(m_plot_opts)

In [ ]:
# BOKEH BACKEND

hv.extension('bokeh')
b_fig = gv.WMTS(STAMEN_TERRAIN) * gv_ds.to(gv.Image, ['lon', 'lat']) * COASTLINES * GRATICLES
         
def adjust_plot(plot, element):
    p = plot.handles['plot']
    for object in p.renderers:
        if isinstance(object, GlyphRenderer):
            if isinstance(object.glyph, Patches):
                object.glyph.fill_alpha = 0
    
b_plot_opts = {'Image': {'style': dict(cmap=CMAP),
                          'plot': dict(title_format=TITLE, width=600, height=600)},
             'Feature': {'style': dict(),
                          'plot': dict(apply_ranges=False, finalize_hooks=[adjust_plot])}
              }             
b_fig.opts(b_plot_opts)